## Quantum Neural Network on $4 \times 4$ downsampled Binary (3, 6) MNIST  
Sebastian Molina  
smolinad@unal.edu.co

Implementation of the paper

[Farhi et. al., Classification with Quantum Neural Networks
on Near Term Processors](https://arxiv.org/pdf/1802.06002.pdf)




In [1]:
!pip install tensorcircuit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.3/364.3 kB 23.9 MB/s eta 0:00:00


In [3]:
from functools import partial
import numpy as np
import tensorflow as tf

import tensorcircuit as tc
from tensorcircuit import keras

In [4]:
tc.set_backend("tensorflow")
tc.set_dtype("complex128")

img_height = 3
n = img_height**2 #Number of qubits and pixels in the downsampled image
nlayers = 3
nsamples = 4000

### MNIST data Preprocessing

In [111]:
import collections

"""
Module for MNIST Digits Dataset preprocessing.
https://www.tensorflow.org/quantum/tutorials/mnist

Python 3.10.11
"""

def filter_by_classes(x, y, classes=[3,6]):
    """
    Function that filters the MNIST Digits Dataset and returns samples on 'classes'.
    Parameters:
        x: Sample images.
        y: Sample labels.
        classes: List of classes to filter.
    Returns:
        x: x filtered by 'classes'.
        y: x filtered by 'classes'.
    """
    if not all(np.isin(classes, range(0, 10))):
        return ValueError("Classes must be a list of digits (0-9).")
    x, y = x[np.isin(y, classes)], y[np.isin(y, classes)]
    if len(classes)==2:
        return x, y==classes[-1]
    else:
        return x, y

def remove_contradicting(xs, ys):

    mapping = collections.defaultdict(set)
    orig_x = {}
    # Determine the set of labels for each unique image:
    for x,y in zip(xs,ys):
       orig_x[tuple(x.flatten())] = x
       mapping[tuple(x.flatten())].add(y)

    new_x = []
    new_y = []
    for flatten_x in mapping:
      x = orig_x[flatten_x]
      labels = mapping[flatten_x]
      if len(labels) == 1:
          new_x.append(x)
          new_y.append(next(iter(labels)))
      else:
          # Throw out images that match more than one label.
          pass

    num_uniq_3 = sum(1 for value in mapping.values() if len(value) == 1 and True in value)
    num_uniq_6 = sum(1 for value in mapping.values() if len(value) == 1 and False in value)
    num_uniq_both = sum(1 for value in mapping.values() if len(value) == 2)

    print("Number of unique images:", len(mapping.values()))
    print("Number of unique 3s: ", num_uniq_3)
    print("Number of unique 6s: ", num_uniq_6)
    print("Number of unique contradicting labels (both 3 and 6): ", num_uniq_both)
    print()
    print("Initial number of images: ", len(xs))
    print("Remaining non-contradicting unique images: ", len(new_x))

    return np.array(new_x), np.array(new_y)

def preprocess_mnist_digits(classes=[3,6]):
    """"
    Function that downloads the MNIST Digits dataset with TensorFlow and performs the following tasks:
        1. Normalizes pixel values from (0, 255) to (0, 1).
        2. By default, returns only 2 classes of digits for classification (this can be deactivated or modified by the 'classes' parameter).
        3. Resizes samples to 4x4 images.
        4. Removes samples that belong to multiple classes simultaneously.
        5. Converts images to binary."
    Parameters:
    Returns:
    """

    # Download dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Rescale the images from [0,255] to the [0.0,1.0] range.
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

    # Filter to get only '3's and '6's
    x_train, y_train = filter_by_classes(x_train, y_train, classes=classes)
    x_test, y_test = filter_by_classes(x_test, y_test, classes=classes)

    print("Number of filtered training examples:", len(x_train))
    print("Number of filtered test examples:", len(x_test))

    # Resize images to 4x4
    x_train_small = tf.image.resize(x_train, (4,4)).numpy()
    x_test_small = tf.image.resize(x_test, (4,4)).numpy()

    x_train_nocon, y_train_nocon = remove_contradicting(x_train_small, y_train)

    THRESHOLD = 0.5

    # Converts non contradicting samples to binary via threshold and converting bool to float.
    x_train_bin = np.array(x_train_nocon > THRESHOLD, dtype=np.float32)
    x_test_bin = np.array(x_test_small > THRESHOLD, dtype=np.float32)

    return x_train_bin.reshape(-1, 16), y_train_nocon, x_test_bin.reshape(-1, 16), y_test

X_train, Y_train, X_test, Y_test = preprocess_mnist_digits()
X_train *= np.pi
X_test *= np.pi
Y_train = np.where(Y_train==False, -1, 1)
Y_test = np.where(Y_test==False, -1, 1)

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

Number of filtered training examples: 12049
Number of filtered test examples: 1968
Number of unique images: 10387
Number of unique 3s:  5426
Number of unique 6s:  4912
Number of unique contradicting labels (both 3 and 6):  49

Initial number of images:  12049
Remaining non-contradicting unique images:  10338
(10338, 16) (10338,) (1968, 16) (1968,)


### Quantum variational classifier (Farhi & Neven)
No vmap or vvag optimization. Using vmap runs way faster.

In [135]:
def variational_classifier(x, weights, nlayers):

    c = tc.Circuit(n+1)

    c.x(0)

    for i in range(1, n+1):
      c.rx(i, theta=x[i-1])

    for i in range(nlayers):
        for j in range(1, n+1):
          if i%2 == 0:
            c.exp1(j, 0, unitary=tc.gates._zx_matrix, theta=weights[i, j-1])
          else:
            c.exp1(j, 0, unitary=tc.gates._xx_matrix, theta=weights[i, j-1])

    return tc.backend.real(c.expectation((tc.gates.y(), [0])))


layer = keras.QuantumLayer(partial(variational_classifier, nlayers=nlayers), [(2 * nlayers, n)])

In [138]:
# Keras interface with Keras training paradigm

model = tf.keras.Sequential([layer])

def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)


model.compile(
    loss = tf.keras.losses.Hinge(),
    optimizer=tf.keras.optimizers.legacy.Adam(0.01),
    metrics=["binary_accuracy"],
)

model.fit(X_train, Y_train, batch_size=16, epochs=20)

Epoch 1/20
647/647 [==============================] - 9928s 15s/step - loss: 0.3956 - binary_accuracy: 0.3543
Epoch 2/20
104/647 [===>..........................] - ETA: 2:18:16 - loss: 0.3623 - binary_accuracy: 0.3642

KeyboardInterrupt: ignored

In [142]:
from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test)
accuracy_score(Y_test > 0, Y_pred > 0)

0.9095528455284553